In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [5]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1.

In [6]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [7]:
# df_q1.groupby("prod")["purchase"].sum()
df_q1.groupby("prod")["purchase"].sum().max()

27995166

In [8]:
top_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
top_prod

'P00025442'

In [9]:
df_q1.loc[df_q1["prod"] == top_prod, "job"].value_counts().idxmax()

4

In [10]:
df_q1_sub = df_q1.loc[df_q1["prod"] == top_prod, ]
df_q1_sub.groupby("job")["purchase"].sum().idxmax()

4

### Q2.

In [11]:
df_id1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]]
df_id1 = df_id1.fillna(0).reset_index(drop = True)

In [12]:
df_id1.drop_duplicates().shape

(21, 3)

In [13]:
df_id1["cnt"] = 1
df_id1_agg = df_id1.groupby(["prod_cat1", "prod_cat2", "prod_cat3"])["cnt"].sum()
df_id1_agg = df_id1_agg.reset_index()
df_id1_agg.head(2)

,prod_cat1,prod_cat2,prod_cat3,cnt
0,1,2.0,5.0,1
1,1,2.0,9.0,1


In [14]:
len(df_id1_agg)

21

In [15]:
df_id1["prod_cat1"] = df_id1["prod_cat1"].astype("int").astype("str")
df_id1["prod_cat2"] = df_id1["prod_cat2"].astype("int").astype("str")
df_id1["prod_cat3"] = df_id1["prod_cat3"].astype("int").astype("str")

In [16]:
df_id1.head(2)

,prod_cat1,prod_cat2,prod_cat3,cnt
0,3,0,0,1
1,1,6,14,1


In [17]:
df_id1["prod_cat"] = df_id1["prod_cat1"] + "-" + df_id1["prod_cat2"] + "-" + df_id1["prod_cat3"]

In [18]:
df_id1.head(2)

,prod_cat1,prod_cat2,prod_cat3,cnt,prod_cat
0,3,0,0,1,3-0-0
1,1,6,14,1,1-6-14


In [19]:
df_id1["prod_cat"].nunique()

21

In [20]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [21]:
df_q2 = df.loc[df["age_group"] == "26-35", ]
df_q2 = df_q2[["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.fillna(0).reset_index(drop = True)

In [22]:
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3
0,3,0,1,2.0,0.0
1,5,1,8,0.0,0.0


In [23]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

In [24]:
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0


In [25]:
# df_q2.groupby("user")["prod_cat"].nunique()
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_agg.head(2)

,user,marital,prod_cat
0,3,0,18
1,5,1,43


In [26]:
df_q2["user"].nunique(), len(df_q2_agg)

(2053, 2053)

In [27]:
stat_m0 = df_q2_agg.loc[df_q2_agg["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_agg.loc[df_q2_agg["marital"] == 1, "prod_cat"].mean()

In [28]:
round(abs(stat_m0 - stat_m1), 2)

0.13

### Q3.

In [29]:
df.head(1)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370


In [30]:
df["user"].nunique()

5891

In [31]:
df_q3_user = df.iloc[:, [0, 2, 3, 4, 5, 6]].drop_duplicates()
len(df_q3_user)

5891

In [32]:
df_q3_user.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
4,2,M,55+,16,C,0


In [33]:
df_q3_agg1 = df.groupby("user")["prod"].nunique().reset_index()
df_q3_agg2 = df.groupby("user")["purchase"].sum().reset_index()

In [34]:
df_q3_agg1.head(2)

,user,prod
0,1,35
1,2,77


In [35]:
df_q3_join = df_q3_user.merge(df_q3_agg1, on = "user")
df_q3_join = df_q3_join.merge(df_q3_agg2, on = "user")

In [36]:
df_q3_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [37]:
df_q3_join = df_q3_join.drop(columns = "user")

In [38]:
df_q3_dum = pd.get_dummies(df_q3_join, columns = ["job", "city"])
df_q3_dum.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,F,0-17,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,M,55+,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [39]:
df_q3_dum["gender"] = df_q3_dum["gender"].replace({"M": 1, "F": 0})

In [40]:
df_q3_dum["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [41]:
# dict_repl = {"0-17": 0, ~~~~}
ser_repl = pd.Series(df_q3_dum["age_group"].unique())
ser_repl = ser_repl.sort_values().reset_index(drop = True)
ser_repl = pd.Series(ser_repl.index, index = ser_repl)
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [42]:
pd.crosstab(df_q3_dum["age_group"], df_q3_dum["age_group"].replace(ser_repl))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [43]:
a = df["age_group"].unique()
a.sort()
dict_repl = dict(zip(a, range(len(a))))
dict_repl

{'0-17': 0,
 '18-25': 1,
 '26-35': 2,
 '36-45': 3,
 '46-50': 4,
 '51-55': 5,
 '55+': 6}

In [44]:
df_q3_dum["age_group"] = df_q3_dum["age_group"].replace(ser_repl)

In [45]:
df_q3_dum.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,6,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [46]:
df_q3_dum_nor = MinMaxScaler().fit_transform(df_q3_dum)

In [47]:
model_km = KMeans(n_clusters = 7, random_state = 123)
model_km.fit(df_q3_dum_nor)

C:\Users\hyojin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7, random_state=123)

In [48]:
round(silhouette_score(df_q3_dum_nor, labels = model_km.labels_), 2)

0.18